This notebook currently largely reproduces the code at http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139
(it has been naively ported to Python 3)

In [2]:
from collections import *

def xrange(x):
    return iter(range(x))

def train_char_lm(fname, order=4):
    f = open(fname)
    data = f.read()
    #data = f.readlines()
    # data = file(fname).read()
    f.close()
    
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in xrange(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
        
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm


!wget http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt

In [3]:
lm3 = train_char_lm("data/shakespeare_input.txt", order=3)

In [4]:
lm3['fir']

[('s', 0.49944258639910816),
 ('e', 0.32887402452619846),
 ('m', 0.16722408026755853),
 ('i', 0.0011148272017837235),
 ('k', 0.002229654403567447),
 ('a', 0.0011148272017837235)]

In [5]:
lm4 = train_char_lm("data/shakespeare_input.txt", order=4)

In [6]:
lm4['firs']

[('t', 1.0)]

Let's generate some text

In [11]:
from random import random

def generate_letter(lm, history, order):
  ''' Randomly chooses the next letter using the language model.
  
  Inputs:
    lm: The output from calling train_char_lm.
    history: A sequence of text at least 'order' long.
    order: The length of the n-grams in the language model.
    
  Returns: 
    A letter
  '''
  
  history = history[-order:]
  dist = lm[history]
  x = random()
  for c,v in dist:
    x = x - v
    if x <= 0: return c

In [12]:
def generate_text(lm, order, nletters=500):
  '''Generates a bunch of random text based on the language model.
  
  Inputs:
  lm: The output from calling train_char_lm.
  order: The length of the n-grams in the language model.
  nletters: the number of characters worth of text to generate
  
  Returns: 
    A letter  
  '''
  history = "~" * order
  out = []
  for i in range(nletters):
    c = generate_letter(lm, history, order)
    history = history[-order:] + c
    out.append(c)
  return "".join(out)

In [13]:
lm_2 = train_char_lm("data/shakespeare_input.txt", order=2)
generate_text(lm_2, 2, 500)

"Fir Seettim your he I an and this thim ther clabouty;\nUnabow tho dow:\nThathoraid, I wer reek haft.\n\nProur promplat re my my me;\nLANTIO:\nI'll byse, thande alorromen, an blown! I king they wity thon, th, and hent shour st have cons hatem a not of So not ey, bithall you of bed reat she an, tre an clorte, Give,\nWhim.\n\nPIDARD RING HUBEL:\nNay ant: ther sweds noweete mon re thy se hine. War; I cap ineithervill int a\nyour my by goo he Bromigh even enstrund my the sh-laclown: way's my togul ther prin to "

In [14]:
lm_4 = train_char_lm("data/shakespeare_input.txt", order=4)
generate_text(lm_4, 4, 500)

"First Citizen:\n'Tis proclaimed toothness the done wreath left them, and with my me: go, I pray frame the gall shall his killet us loved at thou tress:\nO you me of thief-Just not fame, looked of the Stay with me.\n\nMARK ANTONIO:\nIn all by the kite depart intrust us, grate,\nOur pure\nNurse and might cold,\nDrew him! dost spirit,\nI put them in brace warliamend the consent for you comes their fit,\nHe's a very gown.\n\nGLOUCESTER:\nThe deparelest day; town!\n\nSIR HUGH EVANS:\nNo long-time's tears:\nHe's dead "

In [16]:
lm_9 = train_char_lm("data/shakespeare_input.txt", order=9)
generate_text(lm_9, 9, 500)

"First Citizen:\nFor you in my dishonour'd dame,\nLord Junius Brutus, is not quickly.\n\nMessenger:\nWell, I had rather that such a place, I multiplying spawn how can I make a voyage, and I'll salute your grace defend us.\n\nTITUS ANDRONICUS:\nI did but trifles, nothing can we call her?\n\nSPEED:\nI tell this evening?\nWhat masquing mates by the story,\nThe court of Venice;\nMy house and at my departure, as you have said, so done, is well.\n\nMARCIUS:\nSir,\nI am thus early for thee: for that. Does he lie at\nthe a"

<B>Change to GM-RKB Data</B>

In [20]:
lm = train_char_lm("data/rkb-mediawiki-20180324-1203.srt.brk.3.txt", order=5)
generate_text(lm, 5, 500)

'15. Though multi-latent description]], where [[subfield]]s, but a [[hypothesis]] at [[frequence Reliable]] is challenge]]s, [[sparsing]] [[methods]] for [[experimenterpretable]], based on [[centrality]] .\nIn additiveness]] the <i>[[minimizing [[algorithm]] has [[low-quality]] are considered a specific changes]] are some [[content regulator]] in the particularly]] plays the [[real]] [[author]] and is [[memory-efficient]]s for [[experiment]], [[Discrete interest]] of [[formation making problem]], '

In [21]:
lm_12 = train_char_lm("data/rkb-mediawiki-20180324-1203.srt.brk.3.txt", order=12)
generate_text(lm_12, 12, 500)

'15. Though one [[man]]’s [[idea]] of [[our approach]] for [[semi-supervised manner]] and present our [[learning algorithm]] does not include [[capitalism]] works effectively [[map]] long [[entity identification]] .\n[[Our results]] comparable [[cost data]] form the [[target]] [[relational data (network)]], and [[harness]] them [[collectively]] to [[estimate / update]] [[the]] [[effectiveness]] and [[performance]] .\nSpecifically, [[we]] consider the [[problem]] of [[efficient]] [[iteration]]s when'

In [19]:
lm_12['[[experiment']

[('a', 0.3909348441926346),
 (']', 0.48725212464589235),
 ('s', 0.0906515580736544),
 ('e', 0.0056657223796034),
 (' ', 0.019830028328611898),
 ('i', 0.0056657223796034)]

----

----